CNN & RNN EMBEDDING

In [112]:
import os, sys
# notebooks/에서 한 단계 위로 올라간 폴더를 PATH에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
from env.custom_maze_env import CustomMazeEnv
from env.get_retina_image import reconstruct

In [113]:
from minigrid.wrappers import RGBImgPartialObsWrapper
base_env = CustomMazeEnv(
    layout_id="c",
    goal_pos=(0,3),
    view_size=5,
    max_steps=250,
    tile_size=32,
    render_mode="rgb_array"
)

env = RGBImgPartialObsWrapper(base_env, tile_size=32)
obs, _ = env.reset(seed=40)
top_down_view = env.render()
fp_view = obs['image'] 

# print("Start:", obs["position"])

# # 4) 원본 goal_pos 읽기 (unwrap)
# print("Goal:", env.unwrapped.goal_pos)

# # 5) 탑다운 렌더링


In [ ]:
from torch import nn
import torch
import math



tensor([[  1.4168,   7.3959,  11.0704,   6.8866,  -3.0817,   1.6848,  -1.4286,
          -0.6006,   2.7093,   4.4311,   1.4266,  -5.0897,   1.1369,  12.0853,
         -11.5176,  -3.0403]], grad_fn=<AddmmBackward0>)

In [ ]:
class RNNpolicy()

Memory Slot

In [ ]:


import os
import yaml
import torch
import random
import time
from torch import nn, optim
from torch.distributions import Categorical
from env.custom_maze_env import CustomMazeEnv
from env.get_retina_image import reconstruct
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
import matplotlib.pyplot as plt
import numpy as np
from IPython import display


#==========================
# 모델 import 및 Load
#==========================

RNNPolicy = None


#==========================
# 메모리 import 및 Load
#==========================

RNNPolicy = None


#==========================
# set_seed
#==========================
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


# 1) 설정 불러오기
cfg = yaml.safe_load(open(os.path.join(project_root, "experiments/config/default.yaml")))
set_seed(cfg["train"]["seed"])

# 2) 환경 생성 & 래핑
base_env = CustomMazeEnv(**cfg["env"])
env = RGBImgPartialObsWrapper(base_env, tile_size=cfg["env"]["tile_size"])

action_dim = env.action_space.n

# 3) 에이전트, 옵티마이저
# policy = RNNPolicy(obs_dim, cfg["agent"]["hidden_size"], action_dim)
# optimizer = optim.Adam(policy.parameters(), lr=cfg["agent"]["learning_rate"])


# tensorboard 준비
# if cfg["logging"]["use_tensorboard"]:
#     from torch.utils.tensorboard import SummaryWriter
#     tb = SummaryWriter(cfg["logging"]["tensorboard_dir"])
# else:
#     tb = None

# 4) 학습 루프
ep = 1

obs, _ = env.reset(seed=cfg["train"]["seed"] + ep)
obs_dim = np.prod(reconstruct(obs["image"], render_chanel=1).shape)  # flatten -> 6ox80
# obs["image"] shape = (tile_size * view_size, tile_size * view_size, 3)
retina = reconstruct(obs["image"], render_chanel=1) # 60 x 80
# full_map = obs["image"] # 160 x 160 x 3


state = torch.from_numpy(retina).float().view(1,1,-1)
hx = torch.zeros(1, 1, cfg["agent"]["hidden_size"])
log_probs = []
rewards = []

In [ ]:
state = torch.from_numpy(retina)[None, None, ...] ## 1 x 1 x 60 x 80
hx = torch.zeros(1, 1, cfg["agent"]["hidden_size"])
cnn_embedder = cnn_embed(
    cnn_hidden_lyrs=[16, 32],
    lin_hidden_lyrs=[32, 64],
    input_img_shape=(60, 80)
    )
state_emd = cnn_embedder(state).unsqueeze(0) # 1 x 1 x stateDim
rnn_lyr = nn.RNN(64, 128, batch_first=True)
state_hidden, _ = rnn_lyr.forward(state_emd, hx) # change state_hidden to hx /// 1 x 1 x hiddenDim 

state_hidden

In [ ]:
class rnn_embed(nn.Module):
    def __init__(self, 
                 rnn_input_dim: int,
                 rnn_hidden_dim: int):
    
        super().__init__()
        self.rnn_lyr = nn.RNN(rnn_input_dim, rnn_hidden_dim)
        self.lin_lyr = nn.Linear(rnn_hidden_dim, rnn_hidden_dim)

    def forward(self, hidden_state, current_obs):

        prev_current = torch.cat([hidden_state, current_obs])
        hidden_state_, _ = self.rnn_lyr(prev_current)
        hidden_state_new = self.lin_lyr(hidden_state_)
        
        return hidden_state_new

torch.Size([1, 1, 60, 80])

In [ ]:
##### 메모리에 저장하는 코드 추가해야함 ###
#     # retina : retina(egocentric view)
#     # action : a
#     # hidden_state
#     # timestep : obs['timestep']
#     # agent_postion : obs['position']
#     # episode_number : ep
#     #========================================

tensor([[ 0.0182,  0.0607,  0.0099, -0.0319, -0.0623,  0.0488, -0.0005,  0.0454,
          0.0232, -0.0800,  0.0049,  0.0527,  0.0591,  0.0574,  0.0757,  0.0410]],
       grad_fn=<AddmmBackward0>)

In [152]:
input1 = torch.randn(1,2,8)
input2 = torch.randn(1,103,8)

torch.cat([input1, input2], axis=1).shape

torch.Size([1, 105, 8])

In [154]:
input2[:, -90:,:].shape

torch.Size([1, 90, 8])

In [165]:
import torch 
import torch.nn 
import math
import pickle

class cnn_embed(nn.Module):
    '''
    input: retina
    output: cnn embedding of retina
    '''
    def __init__(
        self,
        cnn_hidden_lyrs: list = [16, 32],
        lin_hidden_lyrs: list = [32, 64],
        input_img_shape: tuple = (60, 80) # HW
    ):
        ## 0)
        super().__init__()
        
        ## 1) CNN LAYERS
        self.cnn_lyrs = []
        cnn_hidden_lyr_prev = 1 ## TBD iINITIAL CHANNEL SET TO 1
        for cnn_hidden_lyr in cnn_hidden_lyrs:
            conv = nn.Conv2d(in_channels = cnn_hidden_lyr_prev,
                            out_channels = cnn_hidden_lyr,
                            kernel_size=3,
                            stride=1,
                            padding=0)
            self.cnn_lyrs += [conv, nn.ReLU(), nn.MaxPool2d(2,2)]
            cnn_hidden_lyr_prev = cnn_hidden_lyr

        ## 2) LINEAR LAYERS
        self.lin_lyrs = [nn.Flatten()]
        if lin_hidden_lyrs == []:
            pass 
        else:
            output_img_shape = self.get_shape(self.cnn_lyrs, (1,1, *input_img_shape)) ## TBD BATCH
            lin_hidden_lyr_prev = math.prod(output_img_shape)
            for lin_hidden_lyr in lin_hidden_lyrs[:-1]:
                self.lin_lyrs += [
                    nn.Linear(lin_hidden_lyr_prev, lin_hidden_lyr),
                    nn.ReLU()
                ]
                lin_hidden_lyr_prev = lin_hidden_lyr

            self.lin_lyrs.append(nn.Linear(lin_hidden_lyr_prev, lin_hidden_lyrs[-1])) ##TBD CHECK

    def forward(self, input_img, only_output=True):
        inter_img = nn.Sequential(*self.cnn_lyrs)(input_img) 
        output_img = nn.Sequential(*self.lin_lyrs)(inter_img)

        if only_output:
            return output_img
        else:
            return output_img, inter_img

    def get_shape(self, lyrs, input_img_shape):
        with torch.no_grad():
            embedder = nn.Sequential(*lyrs)
            output = embedder(torch.zeros(*input_img_shape))
            output_img_shape = output.shape[1:]
            return output_img_shape
    
class memory_bank():
    def __init__(
            self,
            decay_rate: float=0.8,
            noise_std: float=0.1,
            memory_len: int=100
    ):
        self.memory_bank_org = []
        self.memory_bank_hidden = torch.Tensor()
        self.decay_rate = decay_rate
        self.noise_std = noise_std 
        self.memory_len = memory_len
    
    def update(self, retina, embed_state, hidden_state, action, obs, ep, done)

        ## 1) push memory_slot & save if necessary
        self.push(retina, embed_state, hidden_state, action, obs, ep)
        self.save(ep, done)

        ## 2) update two memory banks 
        self.memory_bank_org.append(self.memory_slot)
        self.memory_bank_hidden = torch.cat([
            self.memory_bank_hidden,
            hidden_state 
        ], axis=1) ## TBD FOR LATER RNN MODIFICATION USED AXIS 1 AGGREGATION
        
        self.memory_bank_hidden = self.memory_bank_hidden[:, -self.memory_len:, :]
        self.add_decay()

        
    def push(self, retina, embed_state, hidden_state, action, obs, ep):
        
        self.memory_slot = {
            'obs': retina,
            'embed_state': embed_state,
            'hidden_state': hidden_state,
            'action': action,
            'timestep': obs['timestep'],
            'position': obs['position'],
            'epi_no': ep
        }

    def save(self, ep, done):
        if done:
            # os.makedirs('results/memory_bank', exist_ok=True) 
            with (f'results/memory_bank/{ep}.pkl') as file:
                pickle.dump(self.memory_bank_org, file)
            self.memory_bank_org = []
    
    def add_decay(self, decay_rate, noise_std):
        
        memory_len_real = self.memory_bank_hidden.size(1)
        for idx in range(memory_len_real):
            memory = self.memory_bank_hidden[:, idx, :]
            
        # self.memory_bank_hidden = add_decay(self.memory_bank_hidden)
        # self.memory_bank_hidden = self.memory_bank_hidden[:-10] 

class


SyntaxError: expected ':' (2518245035.py, line 77)

In [156]:
input1.shape


torch.Size([1, 2, 8])

In [161]:
for idx in range(input1.shape[1]):
    hehe = input1[:,idx,:]

hehe.shape

torch.Size([1, 8])

In [164]:
torch.stack([hehe, hehe], axis=1).shape

torch.Size([1, 2, 8])

In [ ]:
def retrieve(hidden_state_t, memorys):
    cos_smlr_max = 0
    for memory in memorys:
        cos_smlr = nn.CosineSimilarity(hidden_state_t, memory)
        if cos_smlr_max < cos_smlr:
            cos_smlr_max = cos_smlr 
            memory_retrieve = memory
        return memory_retrieve, cos_smlr_max

TRAINING

In [ ]:
import os, sys
# notebooks/에서 한 단계 위로 올라간 폴더를 PATH에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)


import os
import yaml
import torch
import random
import time
from torch import nn, optim
from torch.distributions import Categorical
from env.custom_maze_env import CustomMazeEnv
from env.get_retina_image import reconstruct
from minigrid.wrappers import RGBImgPartialObsWrapper, ImgObsWrapper
import matplotlib.pyplot as plt
import numpy as np
from IPython import display


#==========================
# 모델 import 및 Load
#==========================

RNNPolicy = None


#==========================
# 메모리 import 및 Load
#==========================

RNNPolicy = None


#==========================
# set_seed
#==========================
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


# 1) 설정 불러오기
cfg = yaml.safe_load(open(os.path.join(project_root, "experiments/config/default.yaml")))
set_seed(cfg["train"]["seed"])

# 2) 환경 생성 & 래핑
base_env = CustomMazeEnv(**cfg["env"])
env = RGBImgPartialObsWrapper(base_env, tile_size=cfg["env"]["tile_size"])
obs_dim = np.prod(reconstruct(obs["image"], render_chanel=1).shape)  # flatten -> 6ox80
action_dim = env.action_space.n

# # 3) 에이전트, 옵티마이저
# policy = RNNPolicy(obs_dim, cfg["agent"]["hidden_size"], action_dim)
# optimizer = optim.Adam(policy.parameters(), lr=cfg["agent"]["learning_rate"])


# # tensorboard 준비
# if cfg["logging"]["use_tensorboard"]:
#     from torch.utils.tensorboard import SummaryWriter
#     tb = SummaryWriter(cfg["logging"]["tensorboard_dir"])
# else:
#     tb = None

ep = 1
# for ep in range(1, cfg["train"]["total_episodes"] + 1):
obs, _ = env.reset(seed=cfg["train"]["seed"] + ep)
# obs["image"] shape = (tile_size * view_size, tile_size * view_size, 3)
retina = reconstruct(obs["image"], render_chanel=1)# 60 x 80
# full_map = obs["image"] # 160 x 160 x 3


state = torch.from_numpy(retina).float().view(1,1,-1)
hx = torch.zeros(1, 1, cfg["agent"]["hidden_size"])
log_probs = []
rewards = []


In [ ]:
cnn_embedder

torch.Size([1, 1, 4800])

In [ ]:

# #==========================
# # 에피소드 학습
# #==========================
# done = False
# while not done:
#     logits, hx = policy(state, hx)
#     m = Categorical(logits=logits)
#     a = m.sample()
#     log_probs.append(m.log_prob(a))

#     obs, r, term, trunc, _ = env.step(a.item())
#     rewards.append(r)

#     #======================================
#     ##### 메모리에 저장하는 코드 추가해야함 ###
#     # retina : retina(egocentric view)
#     # action : a
#     # hidden_state
#     # timestep : obs['timestep']
#     # agent_postion : obs['position']
#     # episode_number : ep
#     #========================================

#     # next state
#     retina = reconstruct(obs["image"], render_chanel=1)# 60 x 80
#     state = torch.from_numpy(retina).float().view(1,1,-1)
#     done = term or trunc

#     # #============================
#     # # 에피소드 당 손실 계산 (REINFORCE)
#     # #============================
#     # returns = []
#     # G = 0
#     # for r in reversed(rewards):
#     #     G = r + cfg["agent"]["gamma"] * G
#     #     returns.insert(0, G)
#     # returns = torch.tensor(returns)
#     # returns = (returns - returns.mean()) / (returns.std() + 1e-8)

#     # loss = 0
#     # for lp, G in zip(log_probs, returns):
#     #     loss -= lp * G

#     # optimizer.zero_grad()
#     # loss.backward()
#     # optimizer.step()


In [ ]:
# memory 구조
# time step 단위의 메모리 저장(시간순으로 입력되는 경험의 벡터 시퀀스로 구성)
	
# Memory slot : 각 시점의 경험을 담은 벡터 (obs, action, reward, cue, step)
# mᵢ = concat(
#     obs_embeddingᵢ, #시각 입력의 임베딩 (CNN)
#     actionᵢ,             # 선택한 행동 (one-hot or int)
#     hidden_state_t,   # 해당 시점의 RNN hidden state
#     timestepᵢ,            # 상대 시간 정보
#     절대 좌표,  # 학습에 활용하지는 않음
#     에피소드 번호
# )

# Memory bank : Memory slot 보관함
# 시간이 지남에 따라 메모리 decay 적용 <- 노이즈 추가(노이즈 크기는 하이퍼 파라미터로 설정)
# RNN Encoder : 현재의 관측값+회상값을 인코딩하여 hidden state sequence 생성
# 회상 메모리, 현재 관측값은 gate(parameter)를 거쳐 합 1의 크기로 비율을 조정

#  Attention 기반 Retrieval (회상 메모리 생성)
# -> 현재의 관측값(obs_embedding)의 similarity를 가중치로 메모리의 hidden_state를 합침
# -> weighted sum으로 회상된 memory 벡터 생성
# 현재 관측값 기반 hidden_state +회상기반 hidden_state를 gate를 통해 합친다.
# 합쳐진 hidden_state를 기반으로 action을 예측.

# memory/             
# │   │   ├── slots.py        # Slot 클래스 정의 (obs, Δt, h)  
# │   │   ├── bank.py         # MemoryBank 인터페이스  
# │   │   └── forget.py       # 망각곡선·noise 로직  
# │   │
# │   ├── retrieval/          
# │   │   ├── attention.py    # attention 유사도 함수·key/value  
# │   │   └── gate.py         # Retrieval Gate 모듈  
# │   │
# │   ├── model/              
# │   │   └── rnn_agent.py    # RNN+retrieval 통합 모델 


In [125]:
info

NameError: name 'info' is not defined

In [124]:
obs

{'view': {'image': array([[[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [1, 0, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [2, 5, 0],
          [1, 0, 0]],
  
         [[1, 0, 0],
          [1, 0, 0],
          [1, 0, 0],
          [2, 5, 0],
          [1, 0, 0]],
  
         [[2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0],
          [2, 5, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=uint8),
  'direction': 2,
  'mission': 'reach the green goal square'},
 'timestep': 0,
 'position': (15, 11),
 'image': array([[[146, 146, 146],
         [146, 146, 146],
         [146, 146, 146],
         ...,
         [100, 100, 100],
         [100, 100, 100],
         [100, 100, 100]],
 
        [[146, 146, 146],
         [ 76,  76,  76],
         [ 76,  76,  76],
         ...,
         [  0,   0,   0],
         [  0,   0,   0]

In [ ]:
memory 구조
time step 단위의 메모리 저장(시간순으로 입력되는 경험의 벡터 시퀀스로 구성)
	
Memory slot : 각 시점의 경험을 담은 벡터 (obs, action, reward, cue, step)
mᵢ = concat(
    obs_embeddingᵢ, #시각 입력의 임베딩 (CNN)
    actionᵢ,             # 선택한 행동 (one-hot or int)
    hidden_state_t,   # 해당 시점의 RNN hidden state
    timestepᵢ,            # 상대 시간 정보
    절대 좌표,  # 학습에 활용하지는 않음
    에피소드 번호
)

def step(self, action):
        obs_raw, reward, terminated, truncated, info = super().step(action)

        # goal 도달 시 보상/종료
        if tuple(self.agent_pos) == self.goal_pos:
            reward     = 1.0
            terminated = True
        else:set_matplotlib_formats
            reward     = 0.0

        obs = {
            "view":     obs_raw,
            "timestep": self.step_count,
            "position": tuple(self.agent_pos),
        }
        return obs, reward, terminated, truncated, info
